In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
from torch.profiler import profile, record_function, ProfilerActivity

In [ ]:
# This is a cut down version of:
# https://github.com/karpathy/ng-video-lecture/blob/master/gpt.py
# The idea is to see the 

In [2]:
# hyperparameters
# (my GPU ran out of memory with the default settings - so i am halving all the parameters that affect model size)
# (the model size was 10.788929 M parameters)
# (the message was: OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 4.00 GiB total capacity;
# 3.23 GiB already allocated; 0 bytes free; 3.40 GiB reserved in total by PyTorch)
# If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.
# See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF)
# Even with the reduced params (with 2.715713 M parameters) it ran out of memory until I restarted the kernel
# The training took 16 seconds on my GPU and 220 seconds on my CPU with one hundredth of the batches and half for all the footprints stats.

block_size = 4  ## 64  ## 128  ## 256 # what is the maximum context length for predictions?
max_iters = 1  ## 50  ## 5000  # how many training batches to do
eval_interval = 5  ## 500
learning_rate = 3e-4
eval_iters = 20  ## 200  # for estimate_loss
n_embd = 12  ## 192  ## 384
n_head = 3  # giving head_size = n_embd // n_head = 32
n_layer = 1
dropout = 0.2

In [3]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
f'{vocab_size = }'

'vocab_size = 65'

In [4]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        ##print(f'MHA init: {head_size = }')
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        ##print(f'MHA: {x.shape = }')
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [5]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        ##print(f'FeedFoward: {n_embd = }')
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [6]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        ##print(f'Block: {x.shape = }')
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [7]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        print(f'init: {block_size = }')
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        ##print(f'GPTLanguageModel: {tok_emb.shape = }')
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
        ##print(f'GPTLanguageModel: {pos_emb.shape = }')
        x = tok_emb + pos_emb # (B,T,C)
        print(f'GPTLanguageModel: {x.shape = }')
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

In [8]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        print(f'Head: {x.shape = }')
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (hs,C) @ (B,T,C) -> (B,T,hs)  # cost = hs * T * C
        print(f'Head: {k.shape = }')
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)  # cost = hs * T * T
        ##print(f'Head: {wei.shape = }')
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [9]:
m = GPTLanguageModel()
print(sum(p.numel() for p in m.parameters()), ' parameters')
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        # m(torch.tensor([[3, 4], [3, 4], [3, 4], [3, 4], [3, 4]]))
        m(torch.tensor([[3, 4]]))

init: block_size = 4
3545  parameters
GPTLanguageModel: x.shape = torch.Size([1, 2, 12])
Head: x.shape = torch.Size([1, 2, 12])
Head: k.shape = torch.Size([1, 2, 4])
Head: x.shape = torch.Size([1, 2, 12])
Head: k.shape = torch.Size([1, 2, 4])
Head: x.shape = torch.Size([1, 2, 12])
Head: k.shape = torch.Size([1, 2, 4])


In [10]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=4))

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ----------------------------------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                              Input Shapes  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ----------------------------------------  
            model_inference        38.27%       1.227ms       100.00%       3.206ms       3.206ms             1                                        []  
               aten::linear         2.93%      94.000us        13.29%     426.000us      47.333us             9                 [[1, 2, 12], [4, 12], []]  
               aten::matmul         1.72%      55.000us         9.86%     316.000us      35.111us             9                     [[1, 2, 12], [12, 4]]  
                   aten::mm         5.46%     175.000us         

In [11]:
k = [(x.count, x.input_shapes) for x in prof.key_averages(group_by_input_shape=True) if x.key == 'aten::linear']
k
#### batch_size = 5
#### block_size = T = 2
#### n_embd = e = 12
#### head_size = 4
#[(9, [[1, 2, 12], [4, 12], []]),     # (ditto)
# (1, [[1, 2, 12], [12, 12], [12]]),  # Block ffwd   T * e * e
# (1, [[1, 2, 12], [48, 12], [48]]),  # FeedFoward   T * 4 * e * e
# (1, [[1, 2, 48], [12, 48], [12]]),  # FeedFoward   T * 4 * e * e
# (1, [[1, 2, 12], [65, 12], [65]])]  # initial vocab thing (only non-layer thing)

[(9, [[1, 2, 12], [4, 12], []]),
 (1, [[1, 2, 12], [12, 12], [12]]),
 (1, [[1, 2, 12], [48, 12], [48]]),
 (1, [[1, 2, 48], [12, 48], [12]]),
 (1, [[1, 2, 12], [65, 12], [65]])]

In [12]:
k = [(x.count, x.input_shapes) for x in prof.key_averages(group_by_input_shape=True) if x.key == 'aten::matmul']
k
#[(9, [[1, 2, 12], [12, 4]]),   # q = self.query(x)...  T * e * hs * n_head = T * e * e
# (3, [[1, 2, 4], [1, 4, 2]]),  # wei = q @ k     hs * T * T
# (3, [[1, 2, 2], [1, 2, 4]])]  # out = wei @ v   T * T * hs

[(9, [[1, 2, 12], [12, 4]]),
 (3, [[1, 2, 4], [1, 4, 2]]),
 (3, [[1, 2, 2], [1, 2, 4]])]

In [13]:
12288 / 96

128.0

In [14]:
def cost(L, T, e, hs):
    return L * (10 * T * e * e + 2 * T * T * hs)

In [15]:
cost(96, 2048, 12288, 128) / 1024 / 1024 / 1024 / 1024  # for GPT-3

270.09375